In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

import torch
import numpy as np

from dataset import SliceDataModule, SliceGrappaDataModule
from common.utils import save_reconstructions
from collections import defaultdict
from tqdm import tqdm
from pathlib import Path

In [3]:
HASH = "6e5b9a5f11fa41b8aa5f2fe165fc49a0"
SAVE_DIR = f"reconstructions/{HASH}"

In [1]:
from models import FreezedVarNetNAFNetOL as Model

checkpoint = torch.load(f"mlruns/0/{HASH}/artifacts/checkpoints/latest_checkpoint.pth")
model = Model(**checkpoint["hyper_parameters"]["init_args"])
model.load_state_dict(torch.load(f"mlruns/0/{HASH}/artifacts/checkpoints/latest_checkpoint.pth")["state_dict"])
model = model.cuda()

NameError: name 'torch' is not defined

In [5]:
dm = SliceGrappaDataModule(root="/home/Data")
dm.setup("test")
dm.setup("predict")

In [6]:
work = {
    "public": dm.test_dataloader(),
    "private": dm.predict_dataloader()
}

model.eval()
with torch.no_grad():
    for phase, dataloader in work.items():
        reconstructions = defaultdict(dict)
        print(f"Reconstructing {phase} leaderboard...")
        for mask, kspace, grappa, target, maximum, fnames, slices in tqdm(dataloader):
            output = model(kspace.cuda(non_blocking=True), mask.cuda(non_blocking=True), grappa.cuda(non_blocking=True))
            output = model.image_space_crop(output)
            for i in range(output.shape[0]):
                reconstructions[fnames[i]][slices[i]] = output[i].cpu().numpy()

        for fname in reconstructions:
            reconstructions[fname] = np.stack(
                [reconstructions[fname][slice] for slice in sorted(reconstructions[fname])]
            )
        print(f"Saving reconstructions of {phase} leaderboard...")
        save_reconstructions(reconstructions, Path(f"{SAVE_DIR}/{phase}"))

Reconstructing public leaderboard...


100%|██████████| 984/984 [05:31<00:00,  2.97it/s]


Saving reconstructions of public leaderboard...
Reconstructing private leaderboard...


100%|██████████| 984/984 [05:33<00:00,  2.95it/s]


Saving reconstructions of private leaderboard...


In [7]:
!sh leaderboard_eval.sh {HASH}

Model: 6e5b9a5f11fa41b8aa5f2fe165fc49a0

Leaderboard SSIM : 0.9725
========== Details ==========
Leaderboard SSIM (public): 0.9823
Leaderboard SSIM (private): 0.9628
